Esta notebook se ocupará de realizar todas las peticiones a cada uno de los resources obtenidos en el colab anterior.

No es tan simple como realizar una unica petición, a cada resource, dado que eso solo nos otorgaría 50 registros. 
Es necesario hacer varias peticiones, pidiendo 50 registros en cada una, hasta que se agoten los resultados, o hasta que lleguemos al limite de 10.000 registros.

Instalamos una librería para facilitar el uso de la API de MercadoLibre

In [1]:
pip install git+https://github.com/mercadolibre/python-sdk.git

  Cloning https://github.com/mercadolibre/python-sdk.git to /tmp/pip-req-build-zoha1ehy
  Running command git clone -q https://github.com/mercadolibre/python-sdk.git /tmp/pip-req-build-zoha1ehy
     |████████████████████████████████| 138 kB 8.2 MB/s 
  Created wheel for meli: filename=meli-3.0.0-py3-none-any.whl size=39713 sha256=b3e9ee0d14649e1462d26615ca8609bec86a37674eca89989656635ad8fe3444
  Stored in directory: /tmp/pip-ephem-wheel-cache-x4simzqi/wheels/60/64/2d/4c5c8a03c8a655c3d499ecd487a15dc2e740aed5d6574b6305
Successfully built meli
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
da

Para autenticarnos con la API, se debe acceder al siguiente [link](https://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=96683996985285) , copiar el codigo que aparece en la URL, pegarlo en la celda de abajo, donde dice "code", y ejecutar la celda, para obtener el token, que será el que utilizaremos en cada peticion a la API.


In [2]:
from __future__ import print_function
import time
import meli
from meli.rest import ApiException
from pprint import pprint

configuration = meli.Configuration(
    host = "https://api.mercadolibre.com"
)

with meli.ApiClient() as api_client:
    api_instance = meli.OAuth20Api(api_client)
    grant_type = 'authorization_code' # str
    client_id = '96683996985285' # Your client_id
    client_secret = 'nMeP0YOMz9ZW0ujUdp9MEdV1Spr23vWR' # Your client_secret
    redirect_uri = 'https://www.google.com' # Your redirect_uri
    code = 'TG-61cb3a707b998c001c612aa5-204954233' # The parameter CODE
    refresh_token = 'refresh_token_example' # Your refresh_token
try:
    api_response = api_instance.get_token(grant_type=grant_type, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, code=code, refresh_token=refresh_token)
    access_token = api_response["access_token"]
except ApiException as e:
    print("Exception when calling OAuth20Api->get_token: %s\n" % e)

Encapsulamos la funcion get de MercadoLibre, con algunos parametros de utilidad.

Offset es el indice del primer registro que queremos consultar. Ej: Si es 0, nos traerá el primer registro que la API tiene disponible para el Resource que estamos consultando. 

Limit es la cantidad de registros que nos traerá la API. El maximo es 50.

In [3]:
def get_request(resource, offset=0, limit=50):
    api_instance = meli.RestClientApi(api_client)
    try:
        return api_instance.resource_get(resource+"&offset="+str(offset)+"&limit="+str(limit), access_token)
    except ApiException as e:
      print("Exception when calling RestClientApi->resource_get: %s\n" % e)

Vamos a descargar el archivo de resources del bucket, subido en la notebook anterior. De la misma forma que antes, declaramos dos variables, una con el nombre del proyecto y la otra con el nombre del bucket, y nos autenticamos.

Y luego, copiamos del bucket al filesystem, el archivo resources.pkl

In [4]:
project_id = 'cryptic-opus-335323'
bucket_name = 'bdm-unlu'
from google.colab import auth
auth.authenticate_user()
!gsutil cp gs://{bucket_name}/resources/resources.pkl resources.pkl 

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Copying gs://bdm-unlu/resources/resources.pkl...
/ [1 files][427.8 KiB/427.8 KiB]                                                
Operation completed over 1 objects/427.8 KiB.                                    


Importamos la libreria utilizada anteriormente para leer el archivo en binario, y transformarlo a una lista nuevamente.

In [5]:
import pickle

file_name = "resources.pkl"

open_file = open(file_name, "rb")

resources_with_neighborhood = pickle.load(open_file)

Verificamos que contenga la cantidad de resources que debe tener

In [6]:
len(resources_with_neighborhood)

2577

Y esta es la funcion core de esta notebook. Le llega como parametro un resource, y lo que va a hacer es realizar peticiones a la api, pidiendo de a 50 registros, arrancando por el primero.

Cuando la API le conteste con una cantidad de registros inferior a 50, quiere decir que son los últimos que tiene para entregar, entonces pasa a Falso una variable, y deja de solicitar registros.

Y en el otro momento que esta variable pasa a tener valor Falso, es cuando el offset, llegó a 10.000, que es el máximo que permite MercadoLibre.

Todos los registros que la API le contesta en el mientras tanto, los va agregando a una lista, que va a ser la que devuelva cuando deje de solicitar registros.

In [9]:
def get_results(resource_with_neighborhood):
  results = []
  records_remaining = True;
  offset = 0
  while records_remaining:
    api_response = get_request(resource_with_neighborhood, offset, 50)["results"]
    results.extend(api_response)
    offset += 50
    if len(api_response) < 50 or offset == 10000:
      records_remaining = False
  return results

Una vez codificada la funcion anterior, nos encontramos con 2 problemas, que ya están resueltos pero que vale la pena comentarlos.

1: Haciendo las peticiones secuencialmente, iterando por cada resource, tardaba muchisimo tiempo, por lo que optamos en usar Threads, en total 25 workers, que realizan las peticiones en un tiempo de 3 minutos.

2: Si cada worker guardaba en memoria los resultados que iba obteniendo de la API, desbordaban la memoria RAM de la notebook (12gb), por lo que optamos en guardar los resultados en un archivo, para pasar esa carga al disco, y no a la memoria RAM.

Abrimos el archivo mencionado anteriormente, en modo escritura binaria.

In [10]:
open_file = open("test.pkl", "wb")

Definimos una funcion que agrega al archivo, un resultado, utilizando la libreria pickle.

In [11]:
def append_to_file(result):
  pickle.dump(result, open_file)

Este bloque de código levanta los 25 workers, cuyo punto de procesamiento es la funcion core mencionada anteriormente.

Se agregan a una cola cada uno de los resources, se levantan 25 threads con un worker cada uno, que consumen de la cola y realizan las peticiones utilizando la funcion get_results(), y guardan los resultados en el archivo.
Además, para verificar, llevan un conteo de la cantidad de registros consultados.

In [12]:
import queue, time, urllib.request
from threading import Thread

def perform_web_requests(addresses, no_workers):
    class Worker(Thread):
        def __init__(self, request_queue):
            Thread.__init__(self)
            self.queue = request_queue
            self.errors = []
            self.counter = 0

        def run(self):
            while True:
                content = self.queue.get()
                if content == "":
                    break
                try:
                  results = get_results(content)
                  self.counter += len(results)
                  append_to_file(results)
                except Exception as e:
                  self.errors.append(e)
                self.queue.task_done()

    q = queue.Queue()
    for url in addresses:
        q.put(url)

    workers = []
    for _ in range(no_workers):
        worker = Worker(q)
        worker.start()
        workers.append(worker)

    for _ in workers:
        q.put("")

    for worker in workers:
        worker.join()

    e = []
    record_count = 0
    for worker in workers:
        e.extend(worker.errors)
        record_count += worker.counter

    return [e, record_count]

errors, record_count = perform_web_requests(resources_with_neighborhood, 25)

Vemos que no hubo ningún error en los workers

In [13]:
errors

[]

Observamos la cantidad de registros obtenidos por los threads

In [14]:
record_count

308017

In [15]:
open_file.close()

Copiamos el archivo donde se almacenaron todos los registros, al bucket, en el path bdm-unlu/querys/results.pkl. Para lueo consumirlo en la notebook de [Attributes](https://colab.research.google.com/drive/1HzNmYKuCXJXwCriNWZs01-j2mHe8iveG)

In [16]:
!gsutil cp test.pkl gs://{bucket_name}/querys/results.pkl

Copying file://test.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\
Operation completed over 1 objects/1.4 GiB.                                      
